In [ ]:
# start coding here

In [ ]:
import pandas as pd
import anndata
import numpy as np

In [ ]:
df = pd.read_csv(snakemake.input.matrix_tsv, sep="\t", compression="gzip").T

In [ ]:
np.exp(df)-1

In [ ]:
sample_ids = df.index.map(lambda v: v.split("-")[1])

obs = pd.DataFrame(index=df.index,
                   data={
                       "sample_id": sample_ids,
                       "replicate": pd.Categorical(sample_ids.map(lambda v: v[0])),
                       "condition": pd.Categorical(sample_ids.map(lambda v: {"1": "healthy", "2": "non-inflamed", "3": "inflamed"}[v[1]]))
                       
                   }
                  )

In [ ]:
var = pd.DataFrame(index=df.columns, data={
    "gene_name": df.columns
})

In [ ]:
raw_read_count_mimicks = np.ceil(np.exp(df)-1).astype(np.int32)

adata = anndata.AnnData(X=raw_read_count_mimicks.values, obs=obs, var=var)

In [ ]:
import scanpy as sc

cell_qc, gene_qc = sc.pp.calculate_qc_metrics(adata)

In [ ]:
import seaborn as sns
sns.displot(cell_qc["n_genes_by_counts"])
cell_qc[cell_qc["n_genes_by_counts"] < 100]

No need to filter for n_genes_by_counts

In [ ]:
adata.write_h5ad(snakemake.output[0])